# Sentiment Analysis

Now, we have our cleaned dataset. We can start do our sentiment analysis on the comments about ```VOO``` from Subreddit ```r/ETFs```.

### Import Modules

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import collections

import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import tqdm
import transformers

In [92]:
seed = 8888

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

### Import the datasets

In [93]:
cmts_voo = pd.read_csv('../datasets/cleaned_cmts_voo.csv')
cmts_voo.head()

,author,id,created_utc,permalink,body,score,subreddit
0,coinslinger88,kxwqx9i,2024-04-04 04:28:47,/r/ETFs/comments/1buybes/sso_vs_spyvoo/kxwqx9i/,$VOO is for people who hate money,1,ETFs
1,coinslinger88,kxwr28w,2024-04-04 04:29:32,/r/ETFs/comments/1buvulh/just_starting_brokera...,$VOO is for homeless people,0,ETFs
2,Key-Mark4536,kxsmrdk,2024-04-03 10:17:45,/r/ETFs/comments/1buf5zo/voo/kxsmrdk/,"It's not a bad idea, but it wouldn’t add anyth...",6,ETFs
3,thefreewheeler,kxsjef1,2024-04-03 09:55:36,/r/ETFs/comments/1buf5zo/voo/kxsjef1/,No. VOO and VTI are nearly identical. Either a...,3,ETFs
4,Fun_Grapefruit_3416,kxwl2i1,2024-04-04 03:57:51,/r/ETFs/comments/1buf5zo/voo/kxwl2i1/,No. You already have VOO within VTI and the ta...,3,ETFs


### Using Transformer Model in Sentiment Analysis

We will be using ```BERT-Base-Uncased``` model. 

```BERT```: BERT stands for Bidirectional Encoder Representations from Transformers. It's a groundbreaking model introduced by Google in 2018 that revolutionized the field of natural language processing (NLP). BERT is known for its deep understanding of language context, which it achieves through its transformer architecture.

```Base```: The "base" in "bert-base-uncased" indicates the size of the model. BERT typically comes in two sizes: base and large. The base model is smaller and faster, making it more practical for many applications, though the large model generally performs better on NLP tasks. The base model has 110 million parameters, while the large model has 340 million.

```Uncased```: This specifies that the model was trained on text that has been converted to lowercase, meaning the model does not differentiate between uppercase and lowercase letters. This is in contrast to a "cased" model, which is sensitive to letter casing. For instance, in a cased model, "Hello" and "hello" would be treated differently, whereas they would be treated the same in an uncased model.

The tokenizer we used here is ```AutoTokenizer``` from ```Hugging Face```. 
More detail can check at https://huggingface.co/docs/transformers/v4.39.2/en/autoclass_tutorial#autotokenizer

In [94]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [95]:
sentence_pos = "Stock price of VOO will go higher"
sentence_neg = "Never ever buy VOO"

In [96]:
classifier(sentence_pos), classifier(sentence_neg)

([{'label': 'POSITIVE', 'score': 0.5957298278808594}],
 [{'label': 'NEGATIVE', 'score': 0.9967347979545593}])

In [97]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [98]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [99]:
inputs_pos, inputs_neg = tokenizer(sentence_pos), tokenizer(sentence_neg)

In [100]:
inputs_pos, inputs_neg

({'input_ids': [101, 4518, 3976, 1997, 29536, 2080, 2097, 2175, 3020, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [101, 2196, 2412, 4965, 29536, 2080, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]})

In [101]:
tokenizer.convert_ids_to_tokens([29536, 2080])

['vo', '##o']

In [102]:
pt_batch = tokenizer(
    [sentence_pos, sentence_neg], 
    padding=True, 
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

In [103]:
pt_batch

{'input_ids': tensor([[  101,  4518,  3976,  1997, 29536,  2080,  2097,  2175,  3020,   102],
        [  101,  2196,  2412,  4965, 29536,  2080,   102,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}

In [104]:
pt_outputs = pt_model(**pt_batch)

In [105]:
pt_outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1119,  0.2758],
        [ 3.1458, -2.5754]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [106]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = 'distilbert-base-uncased-finetuned-sst-2-english'

pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def get_label_and_score(output):
    # Apply softmax to output logits
    probabilities = torch.nn.functional.softmax(output.logits, dim=-1)

    # Get the predicted label (0 or 1) based on the highest probability
    label = torch.argmax(probabilities, dim=-1).item()

    # Get the probability score for the predicted label
    score = probabilities[0][label].item()

    return label, score

results = []

for comment in cmts_voo['body']:
    pt_batch = tokenizer(
        str(comment), 
        padding=True, 
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    pt_outputs = pt_model(**pt_batch)
    label, score = get_label_and_score(pt_outputs)
    results.append((label, score))

labels, scores = zip(*results)
cmts_voo['sentiment_label'] = labels
cmts_voo['sentiment_score'] = scores


In [107]:
cmts_voo.head()

,author,id,created_utc,permalink,body,score,subreddit,sentiment_label,sentiment_score
0,coinslinger88,kxwqx9i,2024-04-04 04:28:47,/r/ETFs/comments/1buybes/sso_vs_spyvoo/kxwqx9i/,$VOO is for people who hate money,1,ETFs,0,0.996930
1,coinslinger88,kxwr28w,2024-04-04 04:29:32,/r/ETFs/comments/1buvulh/just_starting_brokera...,$VOO is for homeless people,0,ETFs,0,0.994917
2,Key-Mark4536,kxsmrdk,2024-04-03 10:17:45,/r/ETFs/comments/1buf5zo/voo/kxsmrdk/,"It's not a bad idea, but it wouldn’t add anyth...",6,ETFs,0,0.996812
3,thefreewheeler,kxsjef1,2024-04-03 09:55:36,/r/ETFs/comments/1buf5zo/voo/kxsjef1/,No. VOO and VTI are nearly identical. Either a...,3,ETFs,0,0.978397
4,Fun_Grapefruit_3416,kxwl2i1,2024-04-04 03:57:51,/r/ETFs/comments/1buf5zo/voo/kxwl2i1/,No. You already have VOO within VTI and the ta...,3,ETFs,0,0.999175


In [113]:
result = cmts_voo[['created_utc', 'body', 'sentiment_label', 'sentiment_score']]

result.head()

,created_utc,body,sentiment_label,sentiment_score
0,2024-04-04 04:28:47,$VOO is for people who hate money,0,0.996930
1,2024-04-04 04:29:32,$VOO is for homeless people,0,0.994917
2,2024-04-03 10:17:45,"It's not a bad idea, but it wouldn’t add anyth...",0,0.996812
3,2024-04-03 09:55:36,No. VOO and VTI are nearly identical. Either a...,0,0.978397
4,2024-04-04 03:57:51,No. You already have VOO within VTI and the ta...,0,0.999175
